<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-&quot;дерево-решений&quot;" data-toc-modified-id="Модель-&quot;дерево-решений&quot;-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель "дерево решений"</a></span></li><li><span><a href="#Модель-&quot;случайный-лес&quot;" data-toc-modified-id="Модель-&quot;случайный-лес&quot;-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель "случайный лес"</a></span></li><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Рекомендация тарифов

В компании оператора связи произошло обновление тарифов, старые тарифы переводятся в архив и больше использоваться не будут. Компании необходим инструмент, который будет рекомендовать для каждого пользователя новый тариф, основываясь на особенностях его использования тарифных опций в прошлом. В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифные планы. 

**Цель** - построить модель для задачи классификации, которая выберет подходящий тариф для пользователя. 

Необходимо построить модель с максимально большим значением *accuracy*, по техническому заданию не менее 0.75. Предобработка данных уже проведена.

## Знакомство с данными

In [1]:
# загружаем необходимые библиотеки
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.dummy import DummyClassifier


In [2]:
# загружаем датасет
try:
    df = pd.read_csv('/Users/amirk/Downloads/users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
# изучим датафрейм
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Датасет загружен, в данных нет пропусков, тип данных соответсвует описываемому параметру.

## Разделение данных на выборки

In [5]:
# выделим целевой показатель и параметры

features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [6]:
# разобьем данные на 3 выборки: обучающую, валидационную и тестовую (60%, 20%, 20%).

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=123)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=123)

## Построение моделей

### Модель "дерево решений"

In [7]:
best_tree_model = None
best_accuracy_tree = 0
best_max_depth_tree = 1
best_min_sample_split_tree = 2
for depth in range(1,15):
    for sample_split in range(2,5):
        model_tree = DecisionTreeClassifier(max_depth=depth, min_samples_split=sample_split, random_state=123)
        model_tree.fit(features_train, target_train)
        predictions_valid_tree = model_tree.predict(features_valid)
        accuracy_tree = accuracy_score(target_valid, predictions_valid_tree)
        if best_accuracy_tree < accuracy_tree:
            best_accuracy_tree = accuracy_tree
            best_tree_model = model_tree
            best_max_depth_tree = depth
            best_min_sample_split_tree = sample_split
print (f'Лучшая модель "дерево решений" имеет accuracy = {best_accuracy_tree:.4f}, max_depth = {best_max_depth_tree}, min_sample_split = {best_min_sample_split_tree}.')

Лучшая модель "дерево решений" имеет accuracy = 0.7778, max_depth = 10, min_sample_split = 2.


### Модель "случайный лес"

In [8]:
best_forest_model = None
best_accuracy_forest = 0
best_max_depth_forest = 1
best_min_sample_split_forest = 2
best_n_estimators = 1
for depth in range(1, 10):
    for min_sample in range(2,5):
        for est in range(5,50,5):
            model_forest = RandomForestClassifier(max_depth=depth, 
                                                  min_samples_split=min_sample, 
                                                  n_estimators=est, 
                                                  random_state=123)
            model_forest.fit(features_train, target_train)
            predictions_valid_forest = model_forest.predict(features_valid)
            accuracy_forest = accuracy_score(target_valid, predictions_valid_forest)
            if best_accuracy_forest < accuracy_forest:
                best_accuracy_forest = accuracy_forest
                best_forest_model = model_forest
                best_max_depth_forest = depth
                best_min_sample_split_forest = sample_split
                best_n_estimators = est
print (f'Лучшая модель "случайный лес" имеет accuracy = {best_accuracy_forest:.4f}, max_depth = {best_max_depth_forest}, min_sample_split = {best_min_sample_split_forest}, n_estimators = {best_n_estimators}.')

Лучшая модель "случайный лес" имеет accuracy = 0.7960, max_depth = 9, min_sample_split = 4, n_estimators = 15.


### Модель логистической регрессии

In [9]:
best_reg_model = None
best_accuracy_reg = 0
best_solver = None
best_max_iter = 0
for m_iter in range(100,1100,100):
    model_reg = LogisticRegression(random_state=123, solver='lbfgs', max_iter=m_iter)
    model_reg.fit(features_train, target_train)
    predictions_valid_reg = model_reg.predict(features_valid)
    accuracy_reg = accuracy_score(target_valid, predictions_valid_reg)
    if accuracy_reg > best_accuracy_reg:
        best_accuracy_reg = accuracy_reg
        best_reg_model = model_reg
        best_max_iter = m_iter
print (f'Лучшая модель логистической регрессии имеет accuracy = {best_accuracy_reg:.4f}, max_iter = {best_max_iter}.')

Лучшая модель логистической регрессии имеет accuracy = 0.7247, max_iter = 100.


In [10]:
best_reg_model = None
best_accuracy_reg = 0
best_solver = None
best_max_iter = 0
best_C = 0
for m_iter in range(100,1100,100):
    for c in [1, 10, 100, 1000]:
        model_reg = LogisticRegression(random_state=123, solver='lbfgs', max_iter=m_iter, C=c)
        model_reg.fit(features_train, target_train)
        predictions_valid_reg = model_reg.predict(features_valid)
        accuracy_reg = accuracy_score(target_valid, predictions_valid_reg)
        if accuracy_reg > best_accuracy_reg:
            best_accuracy_reg = accuracy_reg
            best_reg_model = model_reg
            best_max_iter = m_iter
            best_C = c
print (f'Лучшая модель логистической регрессии имеет accuracy = {best_accuracy_reg:.4f}, max_iter = {best_max_iter}, C = {best_C}.')

Лучшая модель логистической регрессии имеет accuracy = 0.7247, max_iter = 100, C = 1.


# Выбор лучшей модели   

В ходе исследования мы построили 3 модели для задачи классификации, где основным параметром качества модели являлось значение **accuracy**.

Лучшая `модель "дерево решений"` имеет **accuracy = 0.7778**, max_depth = 10, min_sample_split = 2.   
Лучшая `модель "случайный лес"` имеет **accuracy = 0.7960**, max_depth = 9, min_sample_split = 4, n_estimators = 15.   
Лучшая `модель логистической регрессии` имеет **accuracy = 0.7247**, max_iter = 100.   

**Таким образом, при построении и обучении моделей на данном датасете, лучшей моделью стала** `модель "случайный лес"`**.**

## Проверка модели на тестовой выборке

In [11]:
# проверяем модель"случайный лес" на тестовой выборке
predictions_test_forest = best_forest_model.predict(features_test)
accuracy_forest = accuracy_score(target_test, predictions_test_forest)
print (f'Модель "случайный лес" на тестовых данных имеет accuracy = {accuracy_forest:.4f}')

Модель "случайный лес" на тестовых данных имеет accuracy = 0.8035


**Лучшая обученная модель ("случайный лес") на тестовых данных получила accuracy = 0.8035.**

## Проверка модели на адекватность

Для проверки адекватности модели классификации необходимо сравнить ее со случайной моделью.

In [12]:
model_random = DummyClassifier(strategy='uniform', random_state=123)
model_random.fit(features_train, target_train)
predictions_random_val = model_random.predict(features_valid)
predictions_random_test = model_random.predict(features_test)

accuracy_random_val = accuracy_score(target_valid, predictions_random_val)
accuracy_random_test = accuracy_score(target_test, predictions_random_test)
print (f'Случайная модель на валидационных данных имеет accuracy = {accuracy_random_val:.4f}')
print (f'Случайная модель на тестовых данных имеет accuracy = {accuracy_random_test:.4f}')

Случайная модель на валидационных данных имеет accuracy = 0.4826
Случайная модель на тестовых данных имеет accuracy = 0.5050


**Accuracy** `случайной модели` на тестовых данных равна **0.5050**. Это значительно меньше выбранной лучшей модели (`"случайный лес"`- **accuracy = 0.8035**). Однако необходимо проверить, есть ли между данными значениями статистически значимые различия.   
Нам необходимо сравнить 2 выборки **predictions_random_test** и **predictions_test_forest** с категориальным типом данных. Для этого применяют хи-квадрат Пирсона. Рассчитаем р-value!


In [13]:
from scipy.stats import chi2_contingency
import numpy as np
data = np.array([[predictions_random_test.sum(), (len(predictions_random_test) - predictions_random_test.sum())],
                [predictions_test_forest.sum(), (len(predictions_test_forest) - predictions_test_forest.sum())]])
statistics = chi2_contingency(data)
if statistics[1] > 0.05:
    print(f'Выбранная модель не адекватна, статистически значимых различий со случайной выборкой нет, p-value = {statistics[1]:.3f}')
else:
    print(f'Выбранная модель адекватна, статистически значимые различия со случайной выборкой выявлены, p-value = {statistics[1]:.3f}')


Выбранная модель адекватна, статистически значимые различия со случайной выборкой выявлены, p-value = 0.000


## Выводы
В ходе исследования мы построили 3 модели для задачи классификации (распределение пользователей по типам тарифов), где основным параметром качества модели являлось значение **accuracy**.   

`Модель "дерево решений"` имеет **accuracy = 0.7778**, max_depth = 10, min_sample_split = 2.   
`Модель "случайный лес"` имеет **accuracy = 0.7960**, max_depth = 9, min_sample_split = 4, n_estimators = 15.   
`Модель логистической регрессии` имеет **accuracy = 0.7247**, max_iter = 100.   

Таким образом, при построении и обучении моделей на данном датасете, лучшей моделью стала `модель "случайный лес"`.   

Лучшая обученная модель (`"случайный лес"`) на тестовых данных также показала лучшие результаты! Accuracy данной модели на тестовой выборке = 0.8035.   

При оценке адекватности разработанной модели проведено ее сранение со случайной моделью. Наша модель показала статистически значимо более высокие результаты качества, р-value = 0.000.